In [72]:
import json
import csv
import pandas as pd
from pandas import DataFrame, Series
import dateutil.parser
from dateutil.tz import gettz
import datetime
import pytz
from functools import reduce
import time

In [2]:
tone_names = ['Sadness', 'Analytical', 'Joy', 'Fear', 'Tentative', 'Anger', 'Confident']
tone_names

['Sadness', 'Analytical', 'Joy', 'Fear', 'Tentative', 'Anger', 'Confident']

In [3]:
#keylogger DF

# local_tz = gettz('America/New_York')

# def utc_to_local(utc_dt):
#     return utc_dt.replace(tzinfo=local_tz)

# def roundTime(dt):
#     dt = dt - datetime.timedelta(minutes=dt.minute, seconds=dt.second, microseconds=dt.microsecond)
#     return dt

with open('../keylogger/logs/log_new.json', 'r') as f:
    keyloggerData = json.load(f)

# tzinfos = { "EDT" : gettz("America/New_York") }

def extract_keyloggerData(data):
    keyloggerResults = []
    keystrokeWordCountResults = []

    for d in data:
#         result = [0]*(len(tone_names) + 6)
        keyloggerResult = [0] * (len(tone_names)+1)
        keystrokeWordCountResult = [0] * 6
        
        tones = d['document_tone']['tones']
        for i in range(len(tones)):
            score = tones[i]['score']
            tone_name = tones[i]['tone_name']
            tone_index = tone_names.index(tone_name)
            keyloggerResult[tone_index] = score            

        keyloggerResult[-1] = d['unix_time']

        keyloggerResults.append(tuple(keyloggerResult))
        
        try:
            keystrokeWordCountResult[0] = d['word_count']
            keystrokeWordCountResult[1] = d['uniqueword_ratio']
            keystrokeWordCountResult[2] = d['backspace_count']
            keystrokeWordCountResult[3] = d['avg_dwelltime']
            keystrokeWordCountResult[4] = d['avg_flighttime']
            keystrokeWordCountResult[5] = d['unix_time']
            
            keystrokeWordCountResults.append(tuple(keystrokeWordCountResult))
            
        except:
            continue
            
    return keyloggerResults, keystrokeWordCountResults

both = extract_keyloggerData(keyloggerData)

keyloggerDF = DataFrame(both[0],
                        columns=[tone_name+"_score" for tone_name in tone_names] + ['time'])


In [4]:
keystrokeWordCountDF = DataFrame(both[1],
                        columns=['word_count', 'uniqueword_ratio', 'backspace_count', 'avg_dwelltime', 'avg_flighttime', 'time'])
keystrokeWordCountDF

,word_count,uniqueword_ratio,backspace_count,avg_dwelltime,avg_flighttime,time
0,42,0.714286,4,0.087153,0.171138,1.527742e+09
1,45,0.911111,55,0.100966,0.208556,1.527969e+09
2,73,0.808219,70,0.139891,0.219618,1.527978e+09
3,16,0.687500,15,0.101219,0.311904,1.527981e+09
4,22,0.909091,49,0.089719,0.281055,1.527992e+09
5,34,0.911765,35,0.116004,0.298266,1.528007e+09
6,8,1.000000,12,0.114056,0.237108,1.528041e+09
7,10,1.000000,2,0.128212,0.281025,1.528053e+09
8,21,0.857143,63,0.077993,0.177798,1.528057e+09
9,100,0.610000,148,0.093515,0.197078,1.528060e+09


In [73]:
#affectiva DF

# local_tz = gettz('America/New_York')

# def utc_to_local(utc_dt):
#     return utc_dt.replace(tzinfo=local_tz)

with open('../getAPIdata/merged_file.json', 'r') as f:
    affectivaData = json.load(f)

for x in range(0, len(affectivaData)):
#         affectivaData[x]['time'] = time.gmtime(affectivaData[x]['time']/ 1e3)
        affectivaData[x]['time'] = affectivaData[x]['time']/ 1e3
        affectivaData[x]['emoji'] = len(affectivaData[x]['emoji'])

#     affectivaData[x]['time'] = utc_to_local(datetime.datetime.fromtimestamp((affectivaData[x]['time']/ 1e3)))


affectivaDF = DataFrame(affectivaData)
# affectivaDF.time = affectivaDF.time.apply(roundTime)
# affectivaDF.time.apply(roundTime)
#affectivaDF.emoji = affectivaDF.emoji.apply(lambda x: ",".join(x))
# del affectivaDF['emoji']
#affectivaDF.emotions = affectivaDF.emotions.apply(lambda x: ",".join(x))
del affectivaDF['emotions']
del affectivaDF['max_attention']
del affectivaDF['min_attention']
del affectivaDF['max_engagement']
del affectivaDF['min_engagement']
del affectivaDF['max_valence']
del affectivaDF['min_valence']
# affectivaDF.time = affectivaDF.time.apply(roundTime)

affectivaDF

,avg_attention,avg_engagement,avg_valence,blinks,emoji,time
0,97.300607,0.089296,-0.003278,4.0,1,1.524779e+09
1,97.171321,24.342674,-1.190739,10.0,4,1.525204e+09
2,98.075239,1.411863,-0.749728,21.0,1,1.525365e+09
3,95.685710,12.829322,-2.966262,17.0,2,1.526346e+09
4,96.475466,4.153910,-4.402774,3.0,1,1.528174e+09
5,84.588569,8.368431,-2.101744,13.0,4,1.529620e+09
6,92.848527,29.437271,-4.706613,7.0,6,1.533246e+09
7,97.659079,17.868846,-6.528832,12.0,3,1.533708e+09
8,96.588857,5.755054,-2.220462,7.0,1,1.533828e+09
9,95.299089,9.000765,4.606062,0.0,2,1.534114e+09


In [75]:
other_affectivaDF = affectivaDF.copy()
other_affectivaDF

,avg_attention,avg_engagement,avg_valence,blinks,emoji,time
0,97.300607,0.089296,-0.003278,4.0,1,1.524779e+09
1,97.171321,24.342674,-1.190739,10.0,4,1.525204e+09
2,98.075239,1.411863,-0.749728,21.0,1,1.525365e+09
3,95.685710,12.829322,-2.966262,17.0,2,1.526346e+09
4,96.475466,4.153910,-4.402774,3.0,1,1.528174e+09
5,84.588569,8.368431,-2.101744,13.0,4,1.529620e+09
6,92.848527,29.437271,-4.706613,7.0,6,1.533246e+09
7,97.659079,17.868846,-6.528832,12.0,3,1.533708e+09
8,96.588857,5.755054,-2.220462,7.0,1,1.533828e+09
9,95.299089,9.000765,4.606062,0.0,2,1.534114e+09


In [83]:
# if(affectivaDF.all() == other_affectivaDF.all()):
#     print("true")
affectivaDF.equals(other_affectivaDF)


True

In [6]:
#mood reporter DF

responsesDF = pd.read_csv("../reporter/responses.tsv", sep='\t', header=0)

timeValues = responsesDF.unix_time.values

def roundUnixTime(timestamp):
    timestamp = timestamp - (timestamp % 3600)
    return timestamp

def convertTime(timestamp):
    timestamp = time.gmtime(timestamp)
    return timestamp

# for x in range(0, len(timeValues)):
#     timestamp = time.gmtime(timeValues[x])
#     print(timestamp)
#     timeValues[x] = timestamp

#     timeValues[x] = utc_to_local(datetime.datetime.fromtimestamp(timeValues[x]))
#     timeValues[x] = dateutil.parser.parse(timeValues[x] + " EDT", tzinfos=tzinfos)


# responsesData.time = timeValues
#responsesData.time = responsesData.time.apply(roundTime)

activity_names = list(set(list(map(lambda x: x.lower().strip(), reduce(lambda x,y: x+y, [x.split(",") for x in list(responsesDF.activity.values)])))))

for activity_name in activity_names:
    responsesDF[activity_name.replace(" ", "_") + "_activity"] = responsesDF.activity.apply(lambda x: activity_name in x.lower())

location_names = list(set(list(map(lambda x: x.lower().strip(), reduce(lambda x,y: x+y, [x.split(",") for x in list(responsesDF.location.values)])))))

def split_locations(locations):
    return list(map(lambda x: x.lower().strip(), locations.split(",")))

for location_name in location_names:
    responsesDF[location_name.replace(" ", "_") + "_location"] = responsesDF.location.apply(lambda x: location_name in split_locations(x))

# responsesDF.time = responsesDF.unix_time.apply(roundUnixTime)
# responsesDF.time = responsesDF.time.apply(convertTime)
responsesDF.time = responsesDF.unix_time

del responsesDF['moodNotes']
del responsesDF['trigger']
del responsesDF['activity']
del responsesDF['location']
del responsesDF['unix_time']



In [7]:
import matplotlib.pyplot as plt
# responsesDF.mood.hist(bins=50)
plt.figure(figsize=(20,10))
plt.plot(responsesDF.time, responsesDF.mood)

In [8]:
responsesDF['mood'].mean()

3.1033333333333335

In [9]:
outputsDF = responsesDF[['time','mood', 'morale', 'stress', 'unique_interactions', 'fatigue', 'compulsions', 'alone']].copy()

def convertUTCToTimestamp(timestamp):
    nanosecondTimestamp = timestamp * 1e9
    timestamp = pd.Timestamp(nanosecondTimestamp, tz='US/Eastern')
    return timestamp

outputsDF.time = outputsDF.time.apply(convertUTCToTimestamp)

# moraleDF = responsesDF[['time','morale']].copy()
# moraleDF.time = moraleDF.time.apply(convertUTCToTimestamp)

# stressDF = responsesDF[['time','stress']].copy()
# stressDF.time = stressDF.time.apply(convertUTCToTimestamp)

# socialDF = responsesDF[['time','unique_interactions']].copy()
# socialDF.time = socialDF.time.apply(convertUTCToTimestamp)

# fatigueDF = responsesDF[['time','fatigue']].copy()
# fatigueDF.time = fatigueDF.time.apply(convertUTCToTimestamp)

# compulsionsDF = responsesDF[['time', 'compulsions']].copy()
# compulsionsDF.time = compulsionsDF.time.apply(convertUTCToTimestamp)

# aloneDF = responsesDF[['time', 'alone']].copy()
# aloneDF.time = aloneDF.time.apply(convertUTCToTimestamp)
outputsDF

,time,mood,morale,stress,unique_interactions,fatigue,compulsions,alone
0,2018-04-09 10:03:18.315716096-04:00,4.0,4.0,3.0,1,1.0,True,True
1,2018-04-09 11:03:48.939009024-04:00,4.0,4.0,3.0,1,1.0,True,True
2,2018-04-09 12:00:17.159026944-04:00,3.0,3.0,2.0,1,1.0,False,True
3,2018-04-09 14:34:53.010916096-04:00,3.0,3.0,2.0,2,4.0,False,True
4,2018-04-09 16:56:31.979720960-04:00,3.0,3.0,2.0,0,3.0,False,True
5,2018-04-09 18:00:51.257328896-04:00,3.0,3.0,2.0,0,4.0,False,True
6,2018-04-09 19:58:39.830408192-04:00,4.0,3.0,2.0,2,4.0,True,False
7,2018-04-10 02:00:04.219219968-04:00,4.0,4.0,2.0,2,2.0,True,False
8,2018-04-10 12:47:00.108283136-04:00,3.0,3.0,3.0,2,3.0,True,False
9,2018-04-10 13:54:55.984233984-04:00,2.0,2.0,3.0,1,4.0,True,False


In [10]:
outputs2DF = responsesDF[['with_friends_activity','schoolwork_activity', 'tv_activity', 'home_location', 'time']].copy()

def convertUTCToTimestamp(timestamp):
    nanosecondTimestamp = timestamp * 1e9
    timestamp = pd.Timestamp(nanosecondTimestamp, tz='US/Eastern')
    return timestamp

outputs2DF.time = outputs2DF.time.apply(convertUTCToTimestamp)
outputs2DF

,with_friends_activity,schoolwork_activity,tv_activity,home_location,time
0,False,True,False,False,2018-04-09 10:03:18.315716096-04:00
1,False,True,False,False,2018-04-09 11:03:48.939009024-04:00
2,False,False,False,False,2018-04-09 12:00:17.159026944-04:00
3,False,False,False,False,2018-04-09 14:34:53.010916096-04:00
4,False,True,False,False,2018-04-09 16:56:31.979720960-04:00
5,False,True,False,False,2018-04-09 18:00:51.257328896-04:00
6,False,True,False,False,2018-04-09 19:58:39.830408192-04:00
7,False,False,False,False,2018-04-10 02:00:04.219219968-04:00
8,False,True,False,False,2018-04-10 12:47:00.108283136-04:00
9,False,False,False,False,2018-04-10 13:54:55.984233984-04:00


In [11]:
keyloggerDF.time = keyloggerDF.time.apply(convertUTCToTimestamp)

In [12]:
# keyloggerMoodDF = pd.merge_asof(moodDF, keyloggerDF, on='time', tolerance=pd.Timedelta('2 hours')).dropna()
# keyloggerMoodDF.reset_index(inplace=True)
# del keyloggerMoodDF['index']
# keyloggerMoodDF

keyloggerDF.sort_values('time', inplace=True)
keyloggerOuputsDF = pd.merge_asof(outputsDF, keyloggerDF, on='time', tolerance=pd.Timedelta('2 hours')).dropna()
keyloggerOuputsDF.reset_index(inplace=True)
del keyloggerOuputsDF['index']
keyloggerOuputsDF

keyloggerOuputs2DF = pd.merge_asof(outputs2DF, keyloggerDF, on='time', tolerance=pd.Timedelta('2 hours')).dropna()
keyloggerOuputs2DF.reset_index(inplace=True)
del keyloggerOuputs2DF['index']

In [13]:
keystrokeWordCountDF.time = keystrokeWordCountDF.time.apply(convertUTCToTimestamp)
keystrokeWordCountDF.sort_values('time', inplace=True)
keystrokeWordCountOuputsDF = pd.merge_asof(outputsDF, keystrokeWordCountDF, on='time', tolerance=pd.Timedelta('2 hours')).dropna()
keystrokeWordCountOuputsDF.reset_index(inplace=True)
del keystrokeWordCountOuputsDF['index']

keystrokeWordCountOuputs2DF = pd.merge_asof(outputs2DF, keystrokeWordCountDF, on='time', tolerance=pd.Timedelta('2 hours')).dropna()
keystrokeWordCountOuputs2DF.reset_index(inplace=True)
del keystrokeWordCountOuputs2DF['index']

In [14]:
affectivaDF.time = affectivaDF.time.apply(convertUTCToTimestamp)

In [15]:
affectivaDF.sort_values('time', inplace=True)
affectivaOuputsDF = pd.merge_asof(outputsDF, affectivaDF, on='time', tolerance=pd.Timedelta('2 hours')).dropna()
affectivaOuputsDF.reset_index(inplace=True)
del affectivaOuputsDF['index']

affectivaOuputs2DF = pd.merge_asof(outputs2DF, affectivaDF, on='time', tolerance=pd.Timedelta('2 hours')).dropna()
affectivaOuputs2DF.reset_index(inplace=True)
del affectivaOuputs2DF['index']

In [16]:
#productivity DF
from dateutil import parser
import calendar

with open('../getAPIdata/productivity.json', 'r') as f:
    productivityFile = json.load(f)

productivityData = productivityFile['rows']

final_productivityData = [];
UTC = pytz.timezone('UTC')

for x in range(0, len(productivityData)):
#     if(productivityData[x][0] > '2018-04-01T00:00:00'):
    date = parser.parse(productivityData[x][0])
    dateutc = str(date.astimezone(UTC))
    dateutc2 = dateutc[:19] + "UTC"
    newtime = time.strptime(dateutc2, "%Y-%m-%d %H:%M:%S%Z")
    finalTime = calendar.timegm(newtime)
    prod_score = productivityData[x][4]
    final_productivityData.append((finalTime, prod_score))

productivityDF = DataFrame(final_productivityData, columns=['time', 'productivity_score'])
#productivityDF.time = productivityDF.time.apply(roundTime)
# productivityDF.time.apply(roundTime)

productivityDF

,time,productivity_score
0,1519880400,65.36
1,1519884000,95.82
2,1519887600,96.92
3,1519891200,97.36
4,1519912800,54.81
5,1519916400,98.92
6,1519920000,92.29
7,1519923600,49.20
8,1519927200,57.45
9,1519930800,46.54


In [17]:
productivityDF.time = productivityDF.time.apply(convertUTCToTimestamp)

In [18]:
productivityDF.sort_values('time', inplace=True)
productivityOuputsDF = pd.merge_asof(outputsDF, productivityDF, on='time', tolerance=pd.Timedelta('2 hours')).dropna()
productivityOuputsDF.reset_index(inplace=True)
del productivityOuputsDF['index']

productivityOuputs2DF = pd.merge_asof(outputs2DF, productivityDF, on='time', tolerance=pd.Timedelta('2 hours')).dropna()
productivityOuputs2DF.reset_index(inplace=True)
del productivityOuputs2DF['index']

In [91]:
# google fit

def convertTimeToStruct(timestamp):
    timestamp = time.gmtime(timestamp)
    return timestamp


with open('../google_fit/dataset.json', 'r') as f:
    fitData = json.load(f)
    

stepCountCompressed = []

def extract_fitData(fitData):
    current_timestamp = int(fitData[0]['endTimeNanos']) / 1e9
    current_hour = convertTimeToStruct(int(fitData[0]['endTimeNanos']) / 1e9)
    stepCount = 0

    for d in fitData:
        dHour = convertTimeToStruct(int(d['endTimeNanos']) / 1e9)
        if (dHour.tm_year == current_hour.tm_year and dHour.tm_mon == current_hour.tm_mon and dHour.tm_mday == current_hour.tm_mday and dHour.tm_hour == current_hour.tm_hour):
            stepCount += int(d['value'][0]['intVal'])
        else:
            holder = [0] * 2
            holder[0] = current_timestamp
#             holder[1] = current_hour
            holder[1] = stepCount
            
            stepCountCompressed.append(tuple(holder))
            stepCount = 0

            current_hour = convertTimeToStruct(int(d['endTimeNanos']) / 1e9)
            current_timestamp = int(d['endTimeNanos']) / 1e9

            stepCount += int(d['value'][0]['intVal'])

extract_fitData(fitData['point'])


stepCountDF = DataFrame(stepCountCompressed, columns = ['time', 'stepCount'])
# stepCountDF.time = stepCountDF.time.apply(convertTimeToStruct)

stepCountDF

,time,stepCount
0,1.522560e+09,378
1,1.522563e+09,1787
2,1.522566e+09,1403
3,1.522571e+09,1035
4,1.522573e+09,3398
5,1.522577e+09,197
6,1.522583e+09,97
7,1.522585e+09,1617
8,1.522590e+09,89
9,1.522606e+09,7


In [20]:
stepCountDF.time = stepCountDF.time.apply(convertUTCToTimestamp)

In [21]:
stepCountDF.sort_values('time', inplace=True)
stepCountOuputsDF = pd.merge_asof(outputsDF, stepCountDF, on='time', tolerance=pd.Timedelta('1.5 hours')).fillna(0)
stepCountOuputsDF.reset_index(inplace=True)
del stepCountOuputsDF['index']

stepCountOuputs2DF = pd.merge_asof(outputs2DF, stepCountDF, on='time', tolerance=pd.Timedelta('1.5 hours')).fillna(0)
stepCountOuputs2DF.reset_index(inplace=True)
del stepCountOuputs2DF['index']

In [84]:
#tabcounter

with open('../getAPIdata/chromeactivity.json', 'r') as f:
    tabCounterData = json.load(f)

newTabData = []

for key in tabCounterData:
    timestamp = int(key) / 1e3

#     timestamp = time.gmtime(int(key) / 1e3)
#     time = utc_to_local(datetime.datetime.fromtimestamp((int(key)/ 1e3)))
    current_tabCount = tabCounterData[key]['current_tabCount']
    current_windowCount = tabCounterData[key]['current_windowCount']
    tabs_activated = tabCounterData[key]['tabs_activated']
    tabs_created = tabCounterData[key]['tabs_created']
    windows_created = tabCounterData[key]['windows_created']

    newTabData.append([timestamp, current_tabCount, current_windowCount,tabs_activated,tabs_created,windows_created])

tabColumns = ['time', 'current_tabCount', 'current_windowCount','tabs_activated','tabs_created','windows_created']
tabCounterDF = DataFrame(newTabData, columns=tabColumns)
# tabCounterDF.time = tabCounterDF.time.apply(roundTime)

print(len(tabCounterData))

2133


In [23]:
tabCounterDF.time = tabCounterDF.time.apply(convertUTCToTimestamp)

In [24]:
tabCounterDF.sort_values('time', inplace=True)
tabCounterOuputsDF = pd.merge_asof(outputsDF, tabCounterDF, on='time', tolerance=pd.Timedelta('2 hours')).fillna(0)
tabCounterOuputsDF.reset_index(inplace=True)
del tabCounterOuputsDF['index']

tabCounterOuputs2DF = pd.merge_asof(outputs2DF, tabCounterDF, on='time', tolerance=pd.Timedelta('2 hours')).fillna(0)
tabCounterOuputs2DF.reset_index(inplace=True)
del tabCounterOuputs2DF['index']

In [25]:
heartRateDF = pd.read_csv("../webcam-pulse-detector-no_openmdao/heartRate.csv", sep=',', header=0)

# firstHeartRateDF.create_time = firstHeartRateDF.create_time.apply(convertTimeToStruct)
heartRateDF.sort_values('time', inplace=True)
heartRateDF = heartRateDF.reset_index(drop=True)



# heartRateDF = DataFrame(heartRate, columns = ['time', 'heart_rate'])
# heartRateDF.columns = ['time', 'heart_rate']
heartRateDF.time = heartRateDF.time.apply(convertUTCToTimestamp)

heartRateOutputsDF = pd.merge_asof(outputsDF, heartRateDF, on='time', tolerance=pd.Timedelta('2.5 hours')).dropna()
heartRateOutputsDF.reset_index(inplace=True)
del heartRateOutputsDF['index']

heartRateOutputs2DF = pd.merge_asof(outputs2DF, heartRateDF, on='time', tolerance=pd.Timedelta('2.5 hours')).dropna()
heartRateOutputs2DF.reset_index(inplace=True)
del heartRateOutputs2DF['index']
heartRateOutputs2DF

,with_friends_activity,schoolwork_activity,tv_activity,home_location,time,heart_rate
0,False,False,False,True,2018-05-18 16:58:21.794635008-04:00,78.000000
1,False,False,False,True,2018-05-21 16:49:48.449735936-04:00,71.000000
2,False,False,False,True,2018-05-21 17:53:47.788486912-04:00,71.000000
3,False,False,False,True,2018-05-21 22:48:48.799660800-04:00,69.000000
4,False,False,False,True,2018-05-22 14:35:31.013202944-04:00,58.000000
5,False,False,False,True,2018-05-22 15:36:22.071392-04:00,58.000000
6,False,False,False,True,2018-05-22 16:53:15.598140928-04:00,67.000000
7,False,True,False,True,2018-05-22 17:54:15.653515008-04:00,71.000000
8,False,False,False,True,2018-05-22 20:10:34.307340032-04:00,62.000000
9,False,False,False,True,2018-05-22 23:45:01.513551104-04:00,67.000000


In [27]:
def hourOnly(timestamp):
    timestamp = timestamp.hour
#     if (timestamp < 6):
#         timestamp = timestamp + 24
    return timestamp


hourColumn = pd.to_datetime(outputsDF.time, unit='s')
hourDF = DataFrame(hourColumn)
hourDF.sort_values('time', inplace=True)
hourDF['hour'] = hourDF.time.apply(hourOnly)
hourDF

hourOutputsDF = pd.merge_asof(outputsDF, hourDF, on='time', tolerance=pd.Timedelta('1 hours')).dropna()
hourOutputsDF.reset_index(inplace=True)
del hourOutputsDF['index']
# hourMoodDF.time = hourMoodDF.time.apply()

hourOutputs2DF = pd.merge_asof(outputs2DF, hourDF, on='time', tolerance=pd.Timedelta('1 hours')).dropna()
hourOutputs2DF.reset_index(inplace=True)
del hourOutputs2DF['index']

In [30]:
# hourMoodDF.sort_values('time', inplace=True)
# hourMoodOuputsDF = pd.merge_asof(outputsDF, hourMoodDF, on='time', tolerance=pd.Timedelta('1 hours')).dropna()
# hourMoodOuputsDF.reset_index(inplace=True)
# del hourMoodOuputsDF['index']
# hourMoodOuputsDF

In [31]:
productivityFeaturesList = list(productivityDF.columns.drop('time'))

In [35]:
from sklearn import linear_model
from sklearn.svm import SVR

y_productivity_mood = productivityOuputsDF["mood"]
y_productivity_morale = productivityOuputsDF["morale"]
y_productivity_stress = productivityOuputsDF["stress"]
y_productivity_social = productivityOuputsDF["unique_interactions"]
y_productivity_fatigue = productivityOuputsDF["fatigue"]
y_productivity_compulsions = productivityOuputsDF["compulsions"]
y_productivity_alone = productivityOuputsDF["alone"]

X_productivity = productivityOuputsDF[productivityFeaturesList]

In [42]:
from sklearn.model_selection import train_test_split

# productivity-mood
X_productivity_train, X_productivity_test, y_productivity_mood_train, y_productivity_mood_test = train_test_split(
    X_productivity, y_productivity_mood, test_size=0.20, shuffle=False)

productivity_mood_LM = SVR(C=0.1, gamma=0.001)
productivity_mood_LM = productivity_mood_LM.fit(X_productivity_train, y_productivity_mood_train)

# productivity-morale
X_productivity_train, X_productivity_test, y_productivity_morale_train, y_productivity_morale_test = train_test_split(
    X_productivity, y_productivity_morale, test_size=0.20, shuffle=False)

productivity_morale_LM = linear_model.LinearRegression()
productivity_morale_LM = productivity_morale_LM.fit(X_productivity_train, y_productivity_morale_train)

# productivity-stress
X_productivity_train, X_productivity_test, y_productivity_stress_train, y_productivity_stress_test = train_test_split(
    X_productivity, y_productivity_stress, test_size=0.20, shuffle=False)

productivity_stress_LM = linear_model.LinearRegression()
productivity_stress_LM = productivity_stress_LM.fit(X_productivity_train, y_productivity_stress_train)

# productivity-social
X_productivity_train, X_productivity_test, y_productivity_social_train, y_productivity_social_test = train_test_split(
    X_productivity, y_productivity_social, test_size=0.20, shuffle=False)

productivity_social_LM = linear_model.LinearRegression()
productivity_social_LM = productivity_social_LM.fit(X_productivity_train, y_productivity_social_train)

# productivity-fatigue
X_productivity_train, X_productivity_test, y_productivity_fatigue_train, y_productivity_fatigue_test = train_test_split(
    X_productivity, y_productivity_fatigue, test_size=0.20, shuffle=False)

productivity_fatigue_LM = linear_model.LinearRegression()
productivity_fatigue_LM = productivity_fatigue_LM.fit(X_productivity_train, y_productivity_fatigue_train)

# productivity-compulsions
X_productivity_train, X_productivity_test, y_productivity_compulsions_train, y_productivity_compulsions_test = train_test_split(
    X_productivity, y_productivity_compulsions, test_size=0.20, shuffle=False)

productivity_compulsions_LM = linear_model.LinearRegression()
productivity_compulsions_LM = productivity_compulsions_LM.fit(X_productivity_train, y_productivity_compulsions_train)

# productivity-alone
X_productivity_train, X_productivity_test, y_productivity_alone_train, y_productivity_alone_test = train_test_split(
    X_productivity, y_productivity_alone, test_size=0.20, shuffle=False)

productivity_alone_LM = linear_model.LinearRegression()
productivity_alone_LM = productivity_alone_LM.fit(X_productivity_train, y_productivity_alone_train)

print(productivity_mood_LM.score(X_productivity_test, y_productivity_mood_test))
print(productivity_morale_LM.score(X_productivity_test, y_productivity_morale_test))
print(productivity_stress_LM.score(X_productivity_test, y_productivity_stress_test))
print(productivity_social_LM.score(X_productivity_test, y_productivity_social_test))
print(productivity_fatigue_LM.score(X_productivity_test, y_productivity_fatigue_test))
print(productivity_compulsions_LM.score(X_productivity_test, y_productivity_compulsions_test))
print(productivity_alone_LM.score(X_productivity_test, y_productivity_alone_test))

-0.0242782866681
0.0563088485656
0.0497721033735
-0.0114011401493
-0.741632052047
-0.0107771644616
0.0560334715746


In [43]:
y_friends = productivityOuputs2DF["with_friends_activity"]
y_schoolwork = productivityOuputs2DF["schoolwork_activity"]
y_tv = productivityOuputs2DF["tv_activity"]
y_home = productivityOuputs2DF["home_location"]

X_productivity = productivityOuputs2DF[productivityFeaturesList]


# productivity-friends
X_productivity_train, X_productivity_test, y_friends_train, y_friends_test = train_test_split(
    X_productivity, y_friends, test_size=0.20, shuffle=False)

productivity_friends_LM = SVR(C=0.1, gamma=0.001)
productivity_friends_LM = productivity_friends_LM.fit(X_productivity_train, y_friends_train)

# productivity-schoolwork
X_productivity_train, X_productivity_test, y_schoolwork_train, y_schoolwork_test = train_test_split(
    X_productivity, y_schoolwork, test_size=0.20, shuffle=False)

productivity_schoolwork_LM = linear_model.LinearRegression()
productivity_schoolwork_LM = productivity_schoolwork_LM.fit(X_productivity_train, y_schoolwork_train)

# productivity-tv
X_productivity_train, X_productivity_test, y_tv_train, y_tv_test = train_test_split(
    X_productivity, y_tv, test_size=0.20, shuffle=False)

productivity_tv_LM = linear_model.LinearRegression()
productivity_tv_LM = productivity_tv_LM.fit(X_productivity_train, y_tv_train)

# productivity-home
X_productivity_train, X_productivity_test, y_home_train, y_home_test = train_test_split(
    X_productivity, y_home, test_size=0.20, shuffle=False)

productivity_home_LM = linear_model.LinearRegression()
productivity_home_LM = productivity_home_LM.fit(X_productivity_train, y_home_train)


print(productivity_friends_LM.score(X_productivity_test, y_friends_test))
print(productivity_schoolwork_LM.score(X_productivity_test, y_schoolwork_test))
print(productivity_tv_LM.score(X_productivity_test, y_tv_test))
print(productivity_home_LM.score(X_productivity_test, y_home_test))

-0.0240608969345
0.0500642919465
0.0547412137498
0.0434216075804


In [45]:
keyloggerFeaturesList = list(keyloggerDF.columns.drop('time'))

In [46]:
y_mood = keyloggerOuputsDF["mood"]
y_morale = keyloggerOuputsDF["morale"]
y_stress = keyloggerOuputsDF["stress"]
y_social = keyloggerOuputsDF["unique_interactions"]
y_fatigue = keyloggerOuputsDF["fatigue"]
y_compulsions = keyloggerOuputsDF["compulsions"]
y_alone = keyloggerOuputsDF["alone"]

X_keylogger = keyloggerOuputsDF[keyloggerFeaturesList]

# keylogger-mood
X_keylogger_train, X_keylogger_test, y_mood_train, y_mood_test = train_test_split(
    X_keylogger, y_mood, test_size=0.20, shuffle=False)

keylogger_mood_LM = SVR(C=0.1, gamma=0.001)
keylogger_mood_LM = keylogger_mood_LM.fit(X_keylogger_train, y_mood_train)

# keylogger-morale
X_keylogger_train, X_keylogger_test, y_morale_train, y_morale_test = train_test_split(
    X_keylogger, y_morale, test_size=0.20, shuffle=False)

keylogger_morale_LM = SVR(C=0.1, gamma=0.001)
keylogger_morale_LM = keylogger_morale_LM.fit(X_keylogger_train, y_morale_train)

# keylogger-stress
X_keylogger_train, X_keylogger_test, y_stress_train, y_stress_test = train_test_split(
    X_keylogger, y_stress, test_size=0.20, shuffle=False)

keylogger_stress_LM = SVR(C=0.1, gamma=0.001)
keylogger_stress_LM = keylogger_stress_LM.fit(X_keylogger_train, y_stress_train)

# keylogger-social
X_keylogger_train, X_keylogger_test, y_social_train, y_social_test = train_test_split(
    X_keylogger, y_social, test_size=0.20, shuffle=False)

keylogger_social_LM = SVR(C=0.1, gamma=0.001)
keylogger_social_LM = keylogger_social_LM.fit(X_keylogger_train, y_social_train)

# keylogger-fatigue
X_keylogger_train, X_keylogger_test, y_fatigue_train, y_fatigue_test = train_test_split(
    X_keylogger, y_fatigue, test_size=0.20, shuffle=False)

keylogger_fatigue_LM = SVR(C=0.1, gamma=0.001)
keylogger_fatigue_LM = keylogger_fatigue_LM.fit(X_keylogger_train, y_fatigue_train)

# keylogger-compulsions
X_keylogger_train, X_keylogger_test, y_compulsions_train, y_compulsions_test = train_test_split(
    X_keylogger, y_compulsions, test_size=0.20, shuffle=False)

keylogger_compulsions_LM = SVR(C=0.1, gamma=0.001)
keylogger_compulsions_LM = keylogger_compulsions_LM.fit(X_keylogger_train, y_compulsions_train)

# keylogger-alone
X_keylogger_train, X_keylogger_test, y_alone_train, y_alone_test = train_test_split(
    X_keylogger, y_alone, test_size=0.20, shuffle=False)

keylogger_alone_LM = SVR(C=0.1, gamma=0.001)
keylogger_alone_LM = keylogger_alone_LM.fit(X_keylogger_train, y_alone_train)

print(keylogger_mood_LM.score(X_keylogger_test, y_mood_test))
print(keylogger_morale_LM.score(X_keylogger_test, y_morale_test))
print(keylogger_stress_LM.score(X_keylogger_test, y_stress_test))
print(keylogger_social_LM.score(X_keylogger_test, y_social_test))
print(keylogger_fatigue_LM.score(X_keylogger_test, y_fatigue_test))
print(keylogger_compulsions_LM.score(X_keylogger_test, y_compulsions_test))
print(keylogger_alone_LM.score(X_keylogger_test, y_alone_test))

-0.0238600986048
-0.0251212856284
-0.339880621445
-0.148865146113
-0.408096955687
-0.871273537635
-0.401251810806


In [ ]:
y_friends = keyloggerOuputs2DF["with_friends_activity"]
y_schoolwork = keyloggerOuputs2DF["schoolwork_activity"]
y_tv = keyloggerOuputs2DF["tv_activity"]
y_home = keyloggerOuputs2DF["home_location"]

X_keylogger = keyloggerOuputs2DF[keyloggerFeaturesList]


# keylogger-friends
X_keylogger_train, X_keylogger_test, y_friends_train, y_friends_test = train_test_split(
    X_keylogger, y_friends, test_size=0.20, shuffle=False)

keylogger_friends_LM = SVR(C=0.1, gamma=0.001)
keylogger_friends_LM = keylogger_friends_LM.fit(X_keylogger_train, y_friends_train)

# keylogger-schoolwork
X_keylogger_train, X_keylogger_test, y_schoolwork_train, y_schoolwork_test = train_test_split(
    X_keylogger, y_schoolwork, test_size=0.20, shuffle=False)

keylogger_schoolwork_LM = linear_model.LinearRegression()
keylogger_schoolwork_LM = keylogger_schoolwork_LM.fit(X_keylogger_train, y_schoolwork_train)

# keylogger-tv
X_keylogger_train, X_keylogger_test, y_tv_train, y_tv_test = train_test_split(
    X_keylogger, y_tv, test_size=0.20, shuffle=False)

keylogger_tv_LM = SVR(C=0.1, gamma=0.001)
keylogger_tv_LM = keylogger_tv_LM.fit(X_keylogger_train, y_tv_train)

# keylogger-home
X_keylogger_train, X_keylogger_test, y_home_train, y_home_test = train_test_split(
    X_keylogger, y_home, test_size=0.20, shuffle=False)

keylogger_home_LM = linear_model.LinearRegression()
keylogger_home_LM = keylogger_home_LM.fit(X_keylogger_train, y_home_train)


print(keylogger_friends_LM.score(X_keylogger_test, y_friends_test))
print(keylogger_schoolwork_LM.score(X_keylogger_test, y_schoolwork_test))
print(keylogger_tv_LM.score(X_keylogger_test, y_tv_test))
print(keylogger_home_LM.score(X_keylogger_test, y_home_test))

In [59]:
from sklearn.svm import SVR

y_mood = keystrokeWordCountOuputsDF["mood"]
y_morale = keystrokeWordCountOuputsDF["morale"]
y_stress = keystrokeWordCountOuputsDF["stress"]
y_social = keystrokeWordCountOuputsDF["unique_interactions"]
y_fatigue = keystrokeWordCountOuputsDF["fatigue"]
y_compulsions = keystrokeWordCountOuputsDF["compulsions"]
y_alone = keystrokeWordCountOuputsDF["alone"]

keystrokeWordCountFeaturesList = list(keystrokeWordCountDF.columns.drop('time'))
X_keystrokeWordCount = keystrokeWordCountOuputsDF[keystrokeWordCountFeaturesList]

# keystrokeWordCount-mood
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_mood_train, y_mood_test = train_test_split(
    X_keystrokeWordCount, y_mood, test_size=0.20, shuffle=False)

keystrokeWordCount_mood_LM = SVR(C=0.1, gamma=0.001)
keystrokeWordCount_mood_LM = keystrokeWordCount_mood_LM.fit(X_keystrokeWordCount_train, y_mood_train)

# keystrokeWordCount-morale
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_morale_train, y_morale_test = train_test_split(
    X_keystrokeWordCount, y_morale, test_size=0.20, shuffle=False)

keystrokeWordCount_morale_LM = SVR(C=0.1, gamma=0.001)
keystrokeWordCount_morale_LM = keystrokeWordCount_morale_LM.fit(X_keystrokeWordCount_train, y_morale_train)

# keystrokeWordCount-stress
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_stress_train, y_stress_test = train_test_split(
    X_keystrokeWordCount, y_stress, test_size=0.20, shuffle=False)

keystrokeWordCount_stress_LM = SVR(C=0.1, gamma=0.001)
keystrokeWordCount_stress_LM = keystrokeWordCount_stress_LM.fit(X_keystrokeWordCount_train, y_stress_train)

# keystrokeWordCount-social
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_social_train, y_social_test = train_test_split(
    X_keystrokeWordCount, y_social, test_size=0.20, shuffle=False)

keystrokeWordCount_social_LM = SVR(C=0.1, gamma=0.001)
keystrokeWordCount_social_LM = keystrokeWordCount_social_LM.fit(X_keystrokeWordCount_train, y_social_train)

# keystrokeWordCount-fatigue
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_fatigue_train, y_fatigue_test = train_test_split(
    X_keystrokeWordCount, y_fatigue, test_size=0.20, shuffle=False)

keystrokeWordCount_fatigue_LM = SVR(C=0.1, gamma=0.001)
keystrokeWordCount_fatigue_LM = keystrokeWordCount_fatigue_LM.fit(X_keystrokeWordCount_train, y_fatigue_train)

# keystrokeWordCount-compulsions
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_compulsions_train, y_compulsions_test = train_test_split(
    X_keystrokeWordCount, y_compulsions, test_size=0.20, shuffle=False)

keystrokeWordCount_compulsions_LM = linear_model.LinearRegression()
keystrokeWordCount_compulsions_LM = keystrokeWordCount_compulsions_LM.fit(X_keystrokeWordCount_train, y_compulsions_train)

# keystrokeWordCount-alone
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_alone_train, y_alone_test = train_test_split(
    X_keystrokeWordCount, y_alone, test_size=0.20, shuffle=False)

keystrokeWordCount_alone_LM = linear_model.LinearRegression()
keystrokeWordCount_alone_LM = keystrokeWordCount_alone_LM.fit(X_keystrokeWordCount_train, y_alone_train)

print(keystrokeWordCount_mood_LM.score(X_keystrokeWordCount_test, y_mood_test))
print(keystrokeWordCount_morale_LM.score(X_keystrokeWordCount_test, y_morale_test))
print(keystrokeWordCount_stress_LM.score(X_keystrokeWordCount_test, y_stress_test))
print(keystrokeWordCount_social_LM.score(X_keystrokeWordCount_test, y_social_test))
print(keystrokeWordCount_fatigue_LM.score(X_keystrokeWordCount_test, y_fatigue_test))
print(keystrokeWordCount_compulsions_LM.score(X_keystrokeWordCount_test, y_compulsions_test))
print(keystrokeWordCount_alone_LM.score(X_keystrokeWordCount_test, y_alone_test))

-0.267308142477
-0.33277576331
-1.10270429502
-0.187280398749
-0.233243714644
0.00786008695357
-0.142515488777


In [68]:
# y_mood_train.mean()
y_mood_test.mean()

3.210526315789474

In [ ]:
y_friends = keystrokeWordCountOuputs2DF["with_friends_activity"]
y_schoolwork = keystrokeWordCountOuputs2DF["schoolwork_activity"]
y_tv = keystrokeWordCountOuputs2DF["tv_activity"]
y_home = keystrokeWordCountOuputs2DF["home_location"]

X_keystrokeWordCount = keystrokeWordCountOuputs2DF[keystrokeWordCountFeaturesList]


# keystrokeWordCount-friends
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_friends_train, y_friends_test = train_test_split(
    X_keystrokeWordCount, y_friends, test_size=0.20, shuffle=False)

keystrokeWordCount_friends_LM = SVR(C=0.1, gamma=0.001)
keystrokeWordCount_friends_LM = keystrokeWordCount_friends_LM.fit(X_keystrokeWordCount_train, y_friends_train)

# keystrokeWordCount-schoolwork
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_schoolwork_train, y_schoolwork_test = train_test_split(
    X_keystrokeWordCount, y_schoolwork, test_size=0.20, shuffle=False)

keystrokeWordCount_schoolwork_LM = SVR(C=0.1, gamma=0.001)
keystrokeWordCount_schoolwork_LM = keystrokeWordCount_schoolwork_LM.fit(X_keystrokeWordCount_train, y_schoolwork_train)

# keystrokeWordCount-tv
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_tv_train, y_tv_test = train_test_split(
    X_keystrokeWordCount, y_tv, test_size=0.20, shuffle=False)

keystrokeWordCount_tv_LM = SVR(C=0.1, gamma=0.001)
keystrokeWordCount_tv_LM = keystrokeWordCount_tv_LM.fit(X_keystrokeWordCount_train, y_tv_train)

# keystrokeWordCount-home
X_keystrokeWordCount_train, X_keystrokeWordCount_test, y_home_train, y_home_test = train_test_split(
    X_keystrokeWordCount, y_home, test_size=0.20, shuffle=False)

keystrokeWordCount_home_LM = SVR(C=0.1, gamma=0.001)
keystrokeWordCount_home_LM = keystrokeWordCount_home_LM.fit(X_keystrokeWordCount_train, y_home_train)


print(keystrokeWordCount_friends_LM.score(X_keystrokeWordCount_test, y_friends_test))
print(keystrokeWordCount_schoolwork_LM.score(X_keystrokeWordCount_test, y_schoolwork_test))
print(keystrokeWordCount_tv_LM.score(X_keystrokeWordCount_test, y_tv_test))
print(keystrokeWordCount_home_LM.score(X_keystrokeWordCount_test, y_home_test))

In [48]:
y_affectiva_mood = affectivaOuputsDF["mood"]
y_affectiva_morale = affectivaOuputsDF["morale"]
y_affectiva_stress = affectivaOuputsDF["stress"]
y_affectiva_social = affectivaOuputsDF["unique_interactions"]
y_affectiva_fatigue = affectivaOuputsDF["fatigue"]
y_affectiva_compulsions = affectivaOuputsDF["compulsions"]
y_affectiva_alone = affectivaOuputsDF["alone"]

affectivaFeaturesList = list(affectivaDF.columns.drop('time'))
X_affectiva = affectivaOuputsDF[affectivaFeaturesList]

# affectiva-mood
X_affectiva_train, X_affectiva_test, y_affectiva_mood_train, y_affectiva_mood_test = train_test_split(
    X_affectiva, y_affectiva_mood, test_size=0.20, shuffle=False)

affectiva_mood_LM = linear_model.LinearRegression()
affectiva_mood_LM = affectiva_mood_LM.fit(X_affectiva_train, y_affectiva_mood_train)

# affectiva-morale
X_affectiva_train, X_affectiva_test, y_affectiva_morale_train, y_affectiva_morale_test = train_test_split(
    X_affectiva, y_affectiva_morale, test_size=0.20, shuffle=False)

affectiva_morale_LM = linear_model.LinearRegression()
affectiva_morale_LM = affectiva_morale_LM.fit(X_affectiva_train, y_affectiva_morale_train)

# affectiva-stress
X_affectiva_train, X_affectiva_test, y_affectiva_stress_train, y_affectiva_stress_test = train_test_split(
    X_affectiva, y_affectiva_stress, test_size=0.20, shuffle=False)

affectiva_stress_LM = SVR(C=0.1, gamma=0.001)
affectiva_stress_LM = affectiva_stress_LM.fit(X_affectiva_train, y_affectiva_stress_train)

# affectiva-social
X_affectiva_train, X_affectiva_test, y_affectiva_social_train, y_affectiva_social_test = train_test_split(
    X_affectiva, y_affectiva_social, test_size=0.20, shuffle=False)

affectiva_social_LM = linear_model.LinearRegression()
affectiva_social_LM = affectiva_social_LM.fit(X_affectiva_train, y_affectiva_social_train)

# affectiva-fatigue
X_affectiva_train, X_affectiva_test, y_affectiva_fatigue_train, y_affectiva_fatigue_test = train_test_split(
    X_affectiva, y_affectiva_fatigue, test_size=0.20, shuffle=False)

affectiva_fatigue_LM = SVR(C=0.1, gamma=0.001)
affectiva_fatigue_LM = affectiva_fatigue_LM.fit(X_affectiva_train, y_affectiva_fatigue_train)

# affectiva-compulsions
X_affectiva_train, X_affectiva_test, y_affectiva_compulsions_train, y_affectiva_compulsions_test = train_test_split(
    X_affectiva, y_affectiva_compulsions, test_size=0.20, shuffle=False)

affectiva_compulsions_LM = linear_model.LinearRegression()
affectiva_compulsions_LM = affectiva_compulsions_LM.fit(X_affectiva_train, y_affectiva_compulsions_train)

# affectiva-alone
X_affectiva_train, X_affectiva_test, y_affectiva_alone_train, y_affectiva_alone_test = train_test_split(
    X_affectiva, y_affectiva_alone, test_size=0.20, shuffle=False)

affectiva_alone_LM = SVR(C=0.1, gamma=0.001)
affectiva_alone_LM = affectiva_alone_LM.fit(X_affectiva_train, y_affectiva_alone_train)

print(affectiva_mood_LM.score(X_affectiva_test, y_affectiva_mood_test))
print(affectiva_morale_LM.score(X_affectiva_test, y_affectiva_morale_test))
print(affectiva_stress_LM.score(X_affectiva_test, y_affectiva_stress_test))
print(affectiva_social_LM.score(X_affectiva_test, y_affectiva_social_test))
print(affectiva_fatigue_LM.score(X_affectiva_test, y_affectiva_fatigue_test))
print(affectiva_compulsions_LM.score(X_affectiva_test, y_affectiva_compulsions_test))
print(affectiva_alone_LM.score(X_affectiva_test, y_affectiva_alone_test))

0.0728876267603
0.0776465226283
-0.17789026808
0.0199356716285
-0.735147323071
-0.0365842639125
-0.241067488713


In [ ]:
y_friends = affectivaOuputs2DF["with_friends_activity"]
y_schoolwork = affectivaOuputs2DF["schoolwork_activity"]
y_tv = affectivaOuputs2DF["tv_activity"]
y_home = affectivaOuputs2DF["home_location"]

X_affectiva = affectivaOuputs2DF[affectivaFeaturesList]


# affectiva-friends
X_affectiva_train, X_affectiva_test, y_friends_train, y_friends_test = train_test_split(
    X_affectiva, y_friends, test_size=0.20, shuffle=False)

affectiva_friends_LM = SVR(C=0.1, gamma=0.001)
affectiva_friends_LM = affectiva_friends_LM.fit(X_affectiva_train, y_friends_train)

# affectiva-schoolwork
X_affectiva_train, X_affectiva_test, y_schoolwork_train, y_schoolwork_test = train_test_split(
    X_affectiva, y_schoolwork, test_size=0.20, shuffle=False)

affectiva_schoolwork_LM = linear_model.LinearRegression()
affectiva_schoolwork_LM = affectiva_schoolwork_LM.fit(X_affectiva_train, y_schoolwork_train)

# affectiva-tv
X_affectiva_train, X_affectiva_test, y_tv_train, y_tv_test = train_test_split(
    X_affectiva, y_tv, test_size=0.20, shuffle=False)

affectiva_tv_LM = linear_model.LinearRegression()
affectiva_tv_LM = affectiva_tv_LM.fit(X_affectiva_train, y_tv_train)

# affectiva-home
X_affectiva_train, X_affectiva_test, y_home_train, y_home_test = train_test_split(
    X_affectiva, y_home, test_size=0.20, shuffle=False)

affectiva_home_LM = SVR(C=0.1, gamma=0.001)
affectiva_home_LM = affectiva_home_LM.fit(X_affectiva_train, y_home_train)


print(affectiva_friends_LM.score(X_affectiva_test, y_friends_test))
print(affectiva_schoolwork_LM.score(X_affectiva_test, y_schoolwork_test))
print(affectiva_tv_LM.score(X_affectiva_test, y_tv_test))
print(affectiva_home_LM.score(X_affectiva_test, y_home_test))

In [49]:
y_stepCount_mood = stepCountOuputsDF["mood"]
y_stepCount_morale = stepCountOuputsDF["morale"]
y_stepCount_stress = stepCountOuputsDF["stress"]
y_stepCount_social = stepCountOuputsDF["unique_interactions"]
y_stepCount_fatigue = stepCountOuputsDF["fatigue"]
y_stepCount_compulsions = stepCountOuputsDF["compulsions"]
y_stepCount_alone = stepCountOuputsDF["alone"]

stepCountFeaturesList = list(stepCountDF.columns.drop('time'))
X_stepCount = stepCountOuputsDF[stepCountFeaturesList]

# stepCount-mood
X_stepCount_train, X_stepCount_test, y_stepCount_mood_train, y_stepCount_mood_test = train_test_split(
    X_stepCount, y_stepCount_mood, test_size=0.20, shuffle=False)

stepCount_mood_LM = SVR(C=0.1, gamma=0.001)
stepCount_mood_LM = stepCount_mood_LM.fit(X_stepCount_train, y_stepCount_mood_train)

# stepCount-morale
X_stepCount_train, X_stepCount_test, y_stepCount_morale_train, y_stepCount_morale_test = train_test_split(
    X_stepCount, y_stepCount_morale, test_size=0.20, shuffle=False)

stepCount_morale_LM = SVR(C=0.1, gamma=0.001)
stepCount_morale_LM = stepCount_morale_LM.fit(X_stepCount_train, y_stepCount_morale_train)

# stepCount-stress
X_stepCount_train, X_stepCount_test, y_stepCount_stress_train, y_stepCount_stress_test = train_test_split(
    X_stepCount, y_stepCount_stress, test_size=0.20, shuffle=False)

stepCount_stress_LM = SVR(C=0.1, gamma=0.001)
stepCount_stress_LM = stepCount_stress_LM.fit(X_stepCount_train, y_stepCount_stress_train)

# stepCount-social
X_stepCount_train, X_stepCount_test, y_stepCount_social_train, y_stepCount_social_test = train_test_split(
    X_stepCount, y_stepCount_social, test_size=0.20, shuffle=False)

stepCount_social_LM = SVR(C=0.1, gamma=0.001)
stepCount_social_LM = stepCount_social_LM.fit(X_stepCount_train, y_stepCount_social_train)

# stepCount-fatigue
X_stepCount_train, X_stepCount_test, y_stepCount_fatigue_train, y_stepCount_fatigue_test = train_test_split(
    X_stepCount, y_stepCount_fatigue, test_size=0.20, shuffle=False)

stepCount_fatigue_LM = SVR(C=0.1, gamma=0.001)
stepCount_fatigue_LM = stepCount_fatigue_LM.fit(X_stepCount_train, y_stepCount_fatigue_train)

# stepCount-compulsions
X_stepCount_train, X_stepCount_test, y_stepCount_compulsions_train, y_stepCount_compulsions_test = train_test_split(
    X_stepCount, y_stepCount_compulsions, test_size=0.20, shuffle=False)

stepCount_compulsions_LM = SVR(C=0.1, gamma=0.001)
stepCount_compulsions_LM = stepCount_compulsions_LM.fit(X_stepCount_train, y_stepCount_compulsions_train)

# stepCount-alone
X_stepCount_train, X_stepCount_test, y_stepCount_alone_train, y_stepCount_alone_test = train_test_split(
    X_stepCount, y_stepCount_alone, test_size=0.20, shuffle=False)

stepCount_alone_LM = SVR(C=0.1, gamma=0.001)
stepCount_alone_LM = stepCount_alone_LM.fit(X_stepCount_train, y_stepCount_alone_train)

print(stepCount_mood_LM.score(X_stepCount_test, y_stepCount_mood_test))
print(stepCount_morale_LM.score(X_stepCount_test, y_stepCount_morale_test))
print(stepCount_stress_LM.score(X_stepCount_test, y_stepCount_stress_test))
print(stepCount_social_LM.score(X_stepCount_test, y_stepCount_social_test))
print(stepCount_fatigue_LM.score(X_stepCount_test, y_stepCount_fatigue_test))
print(stepCount_compulsions_LM.score(X_stepCount_test, y_stepCount_compulsions_test))
print(stepCount_alone_LM.score(X_stepCount_test, y_stepCount_alone_test))

0.0133143788611
-0.0608289590307
-0.195691263578
0.054696670384
-0.693951164534
-0.0622304756776
0.279303637833


In [ ]:
y_friends = stepCountOuputs2DF["with_friends_activity"]
y_schoolwork = stepCountOuputs2DF["schoolwork_activity"]
y_tv = stepCountOuputs2DF["tv_activity"]
y_home = stepCountOuputs2DF["home_location"]

X_stepCount = stepCountOuputs2DF[stepCountFeaturesList]


# stepCount-friends
X_stepCount_train, X_stepCount_test, y_friends_train, y_friends_test = train_test_split(
    X_stepCount, y_friends, test_size=0.20, shuffle=False)

stepCount_friends_LM = linear_model.LinearRegression()
stepCount_friends_LM = stepCount_friends_LM.fit(X_stepCount_train, y_friends_train)

# stepCount-schoolwork
X_stepCount_train, X_stepCount_test, y_schoolwork_train, y_schoolwork_test = train_test_split(
    X_stepCount, y_schoolwork, test_size=0.20, shuffle=False)

stepCount_schoolwork_LM = linear_model.LinearRegression()
stepCount_schoolwork_LM = stepCount_schoolwork_LM.fit(X_stepCount_train, y_schoolwork_train)

# stepCount-tv
X_stepCount_train, X_stepCount_test, y_tv_train, y_tv_test = train_test_split(
    X_stepCount, y_tv, test_size=0.20, shuffle=False)

stepCount_tv_LM = SVR(C=0.1, gamma=0.001)
stepCount_tv_LM = stepCount_tv_LM.fit(X_stepCount_train, y_tv_train)

# stepCount-home
X_stepCount_train, X_stepCount_test, y_home_train, y_home_test = train_test_split(
    X_stepCount, y_home, test_size=0.20, shuffle=False)

stepCount_home_LM = SVR(C=0.1, gamma=0.001)
stepCount_home_LM = stepCount_home_LM.fit(X_stepCount_train, y_home_train)


print(stepCount_friends_LM.score(X_stepCount_test, y_friends_test))
print(stepCount_schoolwork_LM.score(X_stepCount_test, y_schoolwork_test))
print(stepCount_tv_LM.score(X_stepCount_test, y_tv_test))
print(stepCount_home_LM.score(X_stepCount_test, y_home_test))

In [50]:
y_mood = tabCounterOuputsDF["mood"]
y_morale = tabCounterOuputsDF["morale"]
y_stress = tabCounterOuputsDF["stress"]
y_social = tabCounterOuputsDF["unique_interactions"]
y_fatigue = tabCounterOuputsDF["fatigue"]
y_compulsions = tabCounterOuputsDF["compulsions"]
y_alone = tabCounterOuputsDF["alone"]

tabCounterFeaturesList = list(tabCounterDF.columns.drop('time'))
X_tabCounter = tabCounterOuputsDF[tabCounterFeaturesList]

# tabCounter-mood
X_tabCounter_train, X_tabCounter_test, y_mood_train, y_mood_test = train_test_split(
    X_tabCounter, y_mood, test_size=0.20, shuffle=False)

tabCounter_mood_LM = SVR(C=0.1, gamma=0.001)
tabCounter_mood_LM = tabCounter_mood_LM.fit(X_tabCounter_train, y_mood_train)

# tabCounter-morale
X_tabCounter_train, X_tabCounter_test, y_morale_train, y_morale_test = train_test_split(
    X_tabCounter, y_morale, test_size=0.20, shuffle=False)

tabCounter_morale_LM = SVR(C=0.1, gamma=0.001)
tabCounter_morale_LM = tabCounter_morale_LM.fit(X_tabCounter_train, y_morale_train)

# tabCounter-stress
X_tabCounter_train, X_tabCounter_test, y_stress_train, y_stress_test = train_test_split(
    X_tabCounter, y_stress, test_size=0.20, shuffle=False)

tabCounter_stress_LM = SVR(C=0.1, gamma=0.001)
tabCounter_stress_LM = tabCounter_stress_LM.fit(X_tabCounter_train, y_stress_train)

# tabCounter-social
X_tabCounter_train, X_tabCounter_test, y_social_train, y_social_test = train_test_split(
    X_tabCounter, y_social, test_size=0.20, shuffle=False)

tabCounter_social_LM = SVR(C=0.1, gamma=0.001)
tabCounter_social_LM = tabCounter_social_LM.fit(X_tabCounter_train, y_social_train)

# tabCounter-fatigue
X_tabCounter_train, X_tabCounter_test, y_fatigue_train, y_fatigue_test = train_test_split(
    X_tabCounter, y_fatigue, test_size=0.20, shuffle=False)

tabCounter_fatigue_LM = linear_model.LinearRegression()
tabCounter_fatigue_LM = tabCounter_fatigue_LM.fit(X_tabCounter_train, y_fatigue_train)

# tabCounter-compulsions
X_tabCounter_train, X_tabCounter_test, y_compulsions_train, y_compulsions_test = train_test_split(
    X_tabCounter, y_compulsions, test_size=0.20, shuffle=False)

tabCounter_compulsions_LM = linear_model.LinearRegression()
tabCounter_compulsions_LM = tabCounter_compulsions_LM.fit(X_tabCounter_train, y_compulsions_train)

# tabCounter-alone
X_tabCounter_train, X_tabCounter_test, y_alone_train, y_alone_test = train_test_split(
    X_tabCounter, y_alone, test_size=0.20, shuffle=False)

tabCounter_alone_LM = linear_model.LinearRegression()
tabCounter_alone_LM = tabCounter_alone_LM.fit(X_tabCounter_train, y_alone_train)

print(tabCounter_mood_LM.score(X_tabCounter_test, y_mood_test))
print(tabCounter_morale_LM.score(X_tabCounter_test, y_morale_test))
print(tabCounter_stress_LM.score(X_tabCounter_test, y_stress_test))
print(tabCounter_social_LM.score(X_tabCounter_test, y_social_test))
print(tabCounter_fatigue_LM.score(X_tabCounter_test, y_fatigue_test))
print(tabCounter_compulsions_LM.score(X_tabCounter_test, y_compulsions_test))
print(tabCounter_alone_LM.score(X_tabCounter_test, y_alone_test))

-0.100710852676
-0.0792175126574
-0.615764518081
-0.127096429137
-0.78826292867
-0.087382633244
-0.19740630779


In [ ]:
y_friends = tabCounterOuputs2DF["with_friends_activity"]
y_schoolwork = tabCounterOuputs2DF["schoolwork_activity"]
y_tv = tabCounterOuputs2DF["tv_activity"]
y_home = tabCounterOuputs2DF["home_location"]

X_tabCounter = tabCounterOuputs2DF[tabCounterFeaturesList]


# tabCounter-friends
X_tabCounter_train, X_tabCounter_test, y_friends_train, y_friends_test = train_test_split(
    X_tabCounter, y_friends, test_size=0.20, shuffle=False)

tabCounter_friends_LM = SVR(C=0.1, gamma=0.001)
tabCounter_friends_LM = tabCounter_friends_LM.fit(X_tabCounter_train, y_friends_train)

# tabCounter-schoolwork
X_tabCounter_train, X_tabCounter_test, y_schoolwork_train, y_schoolwork_test = train_test_split(
    X_tabCounter, y_schoolwork, test_size=0.20, shuffle=False)

tabCounter_schoolwork_LM = linear_model.LinearRegression()
tabCounter_schoolwork_LM = tabCounter_schoolwork_LM.fit(X_tabCounter_train, y_schoolwork_train)

# tabCounter-tv
X_tabCounter_train, X_tabCounter_test, y_tv_train, y_tv_test = train_test_split(
    X_tabCounter, y_tv, test_size=0.20, shuffle=False)

tabCounter_tv_LM = linear_model.LinearRegression()
tabCounter_tv_LM = tabCounter_tv_LM.fit(X_tabCounter_train, y_tv_train)

# tabCounter-home
X_tabCounter_train, X_tabCounter_test, y_home_train, y_home_test = train_test_split(
    X_tabCounter, y_home, test_size=0.20, shuffle=False)

tabCounter_home_LM = linear_model.LinearRegression()
tabCounter_home_LM = tabCounter_home_LM.fit(X_tabCounter_train, y_home_train)


print(tabCounter_friends_LM.score(X_tabCounter_test, y_friends_test))
print(tabCounter_schoolwork_LM.score(X_tabCounter_test, y_schoolwork_test))
print(tabCounter_tv_LM.score(X_tabCounter_test, y_tv_test))
print(tabCounter_home_LM.score(X_tabCounter_test, y_home_test))

In [51]:
y_mood = heartRateOutputsDF["mood"]
y_morale = heartRateOutputsDF["morale"]
y_stress = heartRateOutputsDF["stress"]
y_social = heartRateOutputsDF["unique_interactions"]
y_fatigue = heartRateOutputsDF["fatigue"]
y_compulsions = heartRateOutputsDF["compulsions"]
y_alone = heartRateOutputsDF["alone"]

heartRateFeaturesList = list(heartRateDF.columns.drop('time'))
X_heartRate = heartRateOutputsDF[heartRateFeaturesList]

# heartRate-mood
X_heartRate_train, X_heartRate_test, y_mood_train, y_mood_test = train_test_split(
    X_heartRate, y_mood, test_size=0.20, shuffle=False)

heartRate_mood_LM = linear_model.LinearRegression()
heartRate_mood_LM = heartRate_mood_LM.fit(X_heartRate_train, y_mood_train)

# heartRate-morale
X_heartRate_train, X_heartRate_test, y_morale_train, y_morale_test = train_test_split(
    X_heartRate, y_morale, test_size=0.20, shuffle=False)

heartRate_morale_LM = linear_model.LinearRegression()
heartRate_morale_LM = heartRate_morale_LM.fit(X_heartRate_train, y_morale_train)

# heartRate-stress
X_heartRate_train, X_heartRate_test, y_stress_train, y_stress_test = train_test_split(
    X_heartRate, y_stress, test_size=0.20, shuffle=False)

heartRate_stress_LM = linear_model.LinearRegression()
heartRate_stress_LM = heartRate_stress_LM.fit(X_heartRate_train, y_stress_train)

# heartRate-social
X_heartRate_train, X_heartRate_test, y_social_train, y_social_test = train_test_split(
    X_heartRate, y_social, test_size=0.20, shuffle=False)

heartRate_social_LM = linear_model.LinearRegression()
heartRate_social_LM = heartRate_social_LM.fit(X_heartRate_train, y_social_train)

# heartRate-fatigue
X_heartRate_train, X_heartRate_test, y_fatigue_train, y_fatigue_test = train_test_split(
    X_heartRate, y_fatigue, test_size=0.20, shuffle=False)

heartRate_fatigue_LM = SVR(C=0.1, gamma=0.001)
heartRate_fatigue_LM = heartRate_fatigue_LM.fit(X_heartRate_train, y_fatigue_train)

# heartRate-compulsions
X_heartRate_train, X_heartRate_test, y_compulsions_train, y_compulsions_test = train_test_split(
    X_heartRate, y_compulsions, test_size=0.20, shuffle=False)

heartRate_compulsions_LM = linear_model.LinearRegression()
heartRate_compulsions_LM = heartRate_compulsions_LM.fit(X_heartRate_train, y_compulsions_train)

# heartRate-alone
X_heartRate_train, X_heartRate_test, y_alone_train, y_alone_test = train_test_split(
    X_heartRate, y_alone, test_size=0.20, shuffle=False)

heartRate_alone_LM = linear_model.LinearRegression()
heartRate_alone_LM = heartRate_alone_LM.fit(X_heartRate_train, y_alone_train)

print(heartRate_mood_LM.score(X_heartRate_test, y_mood_test))
print(heartRate_morale_LM.score(X_heartRate_test, y_morale_test))
print(heartRate_stress_LM.score(X_heartRate_test, y_stress_test))
print(heartRate_social_LM.score(X_heartRate_test, y_social_test))
print(heartRate_fatigue_LM.score(X_heartRate_test, y_fatigue_test))
print(heartRate_compulsions_LM.score(X_heartRate_test, y_compulsions_test))
print(heartRate_alone_LM.score(X_heartRate_test, y_alone_test))

-0.0719305187059
-0.0390450362619
-1.60137965523
-0.0572973065129
-0.231910477936
-0.0868376746746
-0.0832281882689


In [ ]:
y_friends = heartRateOutputs2DF["with_friends_activity"]
y_schoolwork = heartRateOutputs2DF["schoolwork_activity"]
y_tv = heartRateOutputs2DF["tv_activity"]
y_home = heartRateOutputs2DF["home_location"]

X_heartRate = heartRateOutputs2DF[heartRateFeaturesList]


# heartRate-friends
X_heartRate_train, X_heartRate_test, y_friends_train, y_friends_test = train_test_split(
    X_heartRate, y_friends, test_size=0.20, shuffle=False)

heartRate_friends_LM = SVR(C=0.1, gamma=0.001)
heartRate_friends_LM = heartRate_friends_LM.fit(X_heartRate_train, y_friends_train)

# heartRate-schoolwork
X_heartRate_train, X_heartRate_test, y_schoolwork_train, y_schoolwork_test = train_test_split(
    X_heartRate, y_schoolwork, test_size=0.20, shuffle=False)

heartRate_schoolwork_LM = linear_model.LinearRegression()
heartRate_schoolwork_LM = heartRate_schoolwork_LM.fit(X_heartRate_train, y_schoolwork_train)

# heartRate-tv
X_heartRate_train, X_heartRate_test, y_tv_train, y_tv_test = train_test_split(
    X_heartRate, y_tv, test_size=0.20, shuffle=False)

heartRate_tv_LM = linear_model.LinearRegression()
heartRate_tv_LM = heartRate_tv_LM.fit(X_heartRate_train, y_tv_train)

# heartRate-home
X_heartRate_train, X_heartRate_test, y_home_train, y_home_test = train_test_split(
    X_heartRate, y_home, test_size=0.20, shuffle=False)

heartRate_home_LM = linear_model.LinearRegression()
heartRate_home_LM = heartRate_home_LM.fit(X_heartRate_train, y_home_train)


print(heartRate_friends_LM.score(X_heartRate_test, y_friends_test))
print(heartRate_schoolwork_LM.score(X_heartRate_test, y_schoolwork_test))
print(heartRate_tv_LM.score(X_heartRate_test, y_tv_test))
print(heartRate_home_LM.score(X_heartRate_test, y_home_test))

In [52]:
y_mood = hourOutputsDF["mood"]
y_morale = hourOutputsDF["morale"]
y_stress = hourOutputsDF["stress"]
y_social = hourOutputsDF["unique_interactions"]
y_fatigue = hourOutputsDF["fatigue"]
y_compulsions = hourOutputsDF["compulsions"]
y_alone = hourOutputsDF["alone"]

hourFeaturesList = list(hourDF.columns.drop('time'))
X_hour = hourOutputsDF[hourFeaturesList]

# hour-mood
X_hour_train, X_hour_test, y_mood_train, y_mood_test = train_test_split(
    X_hour, y_mood, test_size=0.20, shuffle=False)

hour_mood_LM = SVR(C=0.1, gamma=0.001)
hour_mood_LM = hour_mood_LM.fit(X_hour_train, y_mood_train)

# hour-morale
X_hour_train, X_hour_test, y_morale_train, y_morale_test = train_test_split(
    X_hour, y_morale, test_size=0.20, shuffle=False)

hour_morale_LM = SVR(C=0.1, gamma=0.001)
hour_morale_LM = hour_morale_LM.fit(X_hour_train, y_morale_train)

# hour-stress
X_hour_train, X_hour_test, y_stress_train, y_stress_test = train_test_split(
    X_hour, y_stress, test_size=0.20, shuffle=False)

hour_stress_LM = SVR(C=0.1, gamma=0.001)
hour_stress_LM = hour_stress_LM.fit(X_hour_train, y_stress_train)

# hour-social
X_hour_train, X_hour_test, y_social_train, y_social_test = train_test_split(
    X_hour, y_social, test_size=0.20, shuffle=False)

hour_social_LM = SVR(C=0.1, gamma=0.001)
hour_social_LM = hour_social_LM.fit(X_hour_train, y_social_train)

# hour-fatigue
X_hour_train, X_hour_test, y_fatigue_train, y_fatigue_test = train_test_split(
    X_hour, y_fatigue, test_size=0.20, shuffle=False)

hour_fatigue_LM = SVR(C=0.1, gamma=0.001)
hour_fatigue_LM = hour_fatigue_LM.fit(X_hour_train, y_fatigue_train)

# hour-compulsions
X_hour_train, X_hour_test, y_compulsions_train, y_compulsions_test = train_test_split(
    X_hour, y_compulsions, test_size=0.20, shuffle=False)

hour_compulsions_LM = SVR(C=0.1, gamma=0.001)
hour_compulsions_LM = hour_compulsions_LM.fit(X_hour_train, y_compulsions_train)

# hour-alone
X_hour_train, X_hour_test, y_alone_train, y_alone_test = train_test_split(
    X_hour, y_alone, test_size=0.20, shuffle=False)

hour_alone_LM = SVR(C=0.1, gamma=0.001)
hour_alone_LM = hour_alone_LM.fit(X_hour_train, y_alone_train)

print(hour_mood_LM.score(X_hour_test, y_mood_test))
print(hour_morale_LM.score(X_hour_test, y_morale_test))
print(hour_stress_LM.score(X_hour_test, y_stress_test))
print(hour_social_LM.score(X_hour_test, y_social_test))
print(hour_fatigue_LM.score(X_hour_test, y_fatigue_test))
print(hour_compulsions_LM.score(X_hour_test, y_compulsions_test))
print(hour_alone_LM.score(X_hour_test, y_alone_test))

-0.0417797786902
-0.00227695154074
-0.139300600219
-0.169954682517
-0.20880836896
-0.149949853389
-0.339349554381


In [ ]:
y_friends = hourOutputs2DF["with_friends_activity"]
y_schoolwork = hourOutputs2DF["schoolwork_activity"]
y_tv = hourOutputs2DF["tv_activity"]
y_home = hourOutputs2DF["home_location"]

X_hour = hourOutputs2DF[hourFeaturesList]


# hour-friends
X_hour_train, X_hour_test, y_friends_train, y_friends_test = train_test_split(
    X_hour, y_friends, test_size=0.20, shuffle=False)

hour_friends_LM = SVR(C=0.1, gamma=0.001)
hour_friends_LM = hour_friends_LM.fit(X_hour_train, y_friends_train)

# hour-schoolwork
X_hour_train, X_hour_test, y_schoolwork_train, y_schoolwork_test = train_test_split(
    X_hour, y_schoolwork, test_size=0.20, shuffle=False)

hour_schoolwork_LM = linear_model.LinearRegression()
hour_schoolwork_LM = hour_schoolwork_LM.fit(X_hour_train, y_schoolwork_train)

# hour-tv
X_hour_train, X_hour_test, y_tv_train, y_tv_test = train_test_split(
    X_hour, y_tv, test_size=0.20, shuffle=False)

hour_tv_LM = linear_model.LinearRegression()
hour_tv_LM = hour_tv_LM.fit(X_hour_train, y_tv_train)

# hour-home
X_hour_train, X_hour_test, y_home_train, y_home_test = train_test_split(
    X_hour, y_home, test_size=0.20, shuffle=False)

hour_home_LM = SVR(C=0.1, gamma=0.001)
hour_home_LM = hour_home_LM.fit(X_hour_train, y_home_train)


print(hour_friends_LM.score(X_hour_test, y_friends_test))
print(hour_schoolwork_LM.score(X_hour_test, y_schoolwork_test))
print(hour_tv_LM.score(X_hour_test, y_tv_test))
print(hour_home_LM.score(X_hour_test, y_home_test))

In [53]:
allOutputsDF = responsesDF[['time','mood', 'morale', 'stress', 'unique_interactions', 'fatigue', 'compulsions', 'alone', 'with_friends_activity','schoolwork_activity', 'tv_activity', 'home_location']].copy()

def convertUTCToTimestamp(timestamp):
    nanosecondTimestamp = timestamp * 1e9
    timestamp = pd.Timestamp(nanosecondTimestamp, tz='US/Eastern')
    return timestamp

allOutputsDF.time = allOutputsDF.time.apply(convertUTCToTimestamp)

y_mood = allOutputsDF["mood"]
y_morale = allOutputsDF["morale"]
y_stress = allOutputsDF["stress"]
y_social = allOutputsDF["unique_interactions"]
y_fatigue = allOutputsDF["fatigue"]
y_compulsions = allOutputsDF["compulsions"]
y_alone = allOutputsDF["alone"]

allOutputsFeaturesList = list(outputs2DF.columns.drop('time'))
X_allOutputs = allOutputsDF[allOutputsFeaturesList]

# allOutputs-mood
X_allOutputs_train, X_allOutputs_test, y_mood_train, y_mood_test = train_test_split(
    X_allOutputs, y_mood, test_size=0.20, shuffle=False)

allOutputs_mood_LM = linear_model.LinearRegression()
allOutputs_mood_LM = allOutputs_mood_LM.fit(X_allOutputs_train, y_mood_train)

# allOutputs-morale
X_allOutputs_train, X_allOutputs_test, y_morale_train, y_morale_test = train_test_split(
    X_allOutputs, y_morale, test_size=0.20, shuffle=False)

allOutputs_morale_LM = linear_model.LinearRegression()
allOutputs_morale_LM = allOutputs_morale_LM.fit(X_allOutputs_train, y_morale_train)

# allOutputs-stress
X_allOutputs_train, X_allOutputs_test, y_stress_train, y_stress_test = train_test_split(
    X_allOutputs, y_stress, test_size=0.20, shuffle=False)

allOutputs_stress_LM = linear_model.LinearRegression()
allOutputs_stress_LM = allOutputs_stress_LM.fit(X_allOutputs_train, y_stress_train)

# allOutputs-social
X_allOutputs_train, X_allOutputs_test, y_social_train, y_social_test = train_test_split(
    X_allOutputs, y_social, test_size=0.20, shuffle=False)

allOutputs_social_LM = linear_model.LinearRegression()
allOutputs_social_LM = allOutputs_social_LM.fit(X_allOutputs_train, y_social_train)

# allOutputs-fatigue
X_allOutputs_train, X_allOutputs_test, y_fatigue_train, y_fatigue_test = train_test_split(
    X_allOutputs, y_fatigue, test_size=0.20, shuffle=False)

allOutputs_fatigue_LM = linear_model.LinearRegression()
allOutputs_fatigue_LM = allOutputs_fatigue_LM.fit(X_allOutputs_train, y_fatigue_train)

# allOutputs-compulsions
X_allOutputs_train, X_allOutputs_test, y_compulsions_train, y_compulsions_test = train_test_split(
    X_allOutputs, y_compulsions, test_size=0.20, shuffle=False)

allOutputs_compulsions_LM = linear_model.LinearRegression()
allOutputs_compulsions_LM = allOutputs_compulsions_LM.fit(X_allOutputs_train, y_compulsions_train)

# allOutputs-alone
X_allOutputs_train, X_allOutputs_test, y_alone_train, y_alone_test = train_test_split(
    X_allOutputs, y_alone, test_size=0.20, shuffle=False)

allOutputs_alone_LM = linear_model.LinearRegression()
allOutputs_alone_LM = allOutputs_alone_LM.fit(X_allOutputs_train, y_alone_train)

print(allOutputs_mood_LM.score(X_allOutputs_test, y_mood_test))
print(allOutputs_morale_LM.score(X_allOutputs_test, y_morale_test))
print(allOutputs_stress_LM.score(X_allOutputs_test, y_stress_test))
print(allOutputs_social_LM.score(X_allOutputs_test, y_social_test))
print(allOutputs_fatigue_LM.score(X_allOutputs_test, y_fatigue_test))
print(allOutputs_compulsions_LM.score(X_allOutputs_test, y_compulsions_test))
print(allOutputs_alone_LM.score(X_allOutputs_test, y_alone_test))

0.340294612487
-0.0128047237439
0.0355620562494
0.220508825607
-0.574404079028
0.0156618208938
0.850749329432


In [ ]:
#train a random forest on the training data
from sklearn import ensemble
producitivityMorale_random_forest = ensemble.RandomForestRegressor(bootstrap=True, max_depth=100, min_samples_leaf=1, min_samples_split=2, n_estimators=800)
producitivityMorale_random_forest = moodHour_random_forest.fit(X_train, y_train)

#predict on the test data and print the results
y_pred_rf = producitivityMorale_random_forest.predict(X_test)
y_pred_rf


# max_depth (depends on # of columns, the smaller the better)
# min_weight_fraction_leaf
# n_estimators
# min_samples_split
# min_samples_leaf

In [ ]:
producitivityMorale_random_forest.score(X_test, y_test)

In [ ]:
import numpy as np
from pprint import pprint

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
# rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = producitivityMorale_random_forest, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 20, 30, 40, 50],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [8, 9, 10, 11, 12],
    'n_estimators': [600, 700, 800, 1000]
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = producitivityMorale_random_forest, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
#train a random forest on the training data
from sklearn import ensemble
producitivityMorale_random_forest = ensemble.RandomForestRegressor(bootstrap=True, max_depth=10, max_features='sqrt', min_samples_leaf=1, min_samples_split=12, n_estimators=700)
producitivityMorale_random_forest = producitivityMorale_random_forest.fit(X_train, y_train)

#predict on the test data and print the results
y_pred_rf = producitivityMorale_random_forest.predict(X_test)
y_pred_rf


# max_depth (depends on # of columns, the smaller the better)
# min_weight_fraction_leaf
# n_estimators
# min_samples_split
# min_samples_leaf

In [ ]:
producitivityMorale_random_forest.score(X_test, y_test)

In [ ]:
from sklearn.svm import SVR

clf = SVR(C=0.1, gamma=0.001)
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

Cs = [0.001, 0.01, 0.1, 1, 10, 100]
gammas = [0.001, 0.01, 0.1, 1, 10, 100]
param_grid = {'C': Cs, 'gamma' : gammas}

svc_grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
# Fit the grid search to the data
svc_grid_search.fit(X_train, y_train)

In [ ]:
svc_grid_search.best_params_

In [61]:
mergedDF1 = pd.merge_asof(keyloggerDF, keystrokeWordCountDF, on='time', tolerance=pd.Timedelta('1 hours')).dropna()
mergedDF1.reset_index(inplace=True)
del mergedDF1['index']

mergedDF2 = pd.merge_asof(affectivaDF, mergedDF1, on='time', direction='nearest', tolerance=pd.Timedelta('2 hours')).dropna()
mergedDF2.reset_index(inplace=True)
del mergedDF2['index']

mergedDF3 = pd.merge_asof(mergedDF2, stepCountDF, on='time', direction='nearest', tolerance=pd.Timedelta('1.5 hours')).fillna(0)
mergedDF3.reset_index(inplace=True)
del mergedDF3['index']

mergedDF4 = pd.merge_asof(mergedDF3, tabCounterDF, on='time', direction='nearest', tolerance=pd.Timedelta('1.5 hours')).dropna()
mergedDF4.reset_index(inplace=True)
del mergedDF4['index']

mergedDF5 = pd.merge_asof(mergedDF4, productivityDF, on='time', direction='nearest', tolerance=pd.Timedelta('1.5 hours')).dropna()
mergedDF5.reset_index(inplace=True)
del mergedDF5['index']

finalMergedDF = pd.merge_asof(mergedDF5, hourOutputsDF, on='time', direction='backward', tolerance=pd.Timedelta('2.5 hours')).dropna()
finalMergedDF.reset_index(inplace=True)
del finalMergedDF['index']

finalMergedDF

,avg_attention,avg_engagement,avg_valence,blinks,emoji,time,Sadness_score,Analytical_score,Joy_score,Fear_score,...,windows_created,productivity_score,mood,morale,stress,unique_interactions,fatigue,compulsions,alone,hour
0,97.914291,1.770243,0.002953,10.0,1,2018-05-30 22:44:26.008000-04:00,0.000000,0.000000,0.000000,0.000000,...,0,85.25,3.0,4.0,0.0,0.0,1.5,True,True,22.0
1,97.740868,0.323982,0.000000,0.0,1,2018-05-30 23:44:34.959000064-04:00,0.000000,0.000000,0.000000,0.000000,...,0,51.70,3.5,3.5,0.0,0.0,3.0,True,True,23.0
2,97.893137,5.792367,-4.189227,9.0,2,2018-06-02 15:52:53.729000192-04:00,0.000000,0.884635,0.000000,0.000000,...,0,20.13,2.5,2.0,3.5,0.0,2.5,True,True,15.0
3,97.768389,11.589639,5.941592,31.0,3,2018-06-02 18:15:26.821000192-04:00,0.548702,0.000000,0.659324,0.000000,...,0,33.40,2.5,2.0,2.0,3.0,1.5,False,True,16.0
4,92.314023,22.378806,-21.662834,6.0,2,2018-06-02 19:15:28.896000-04:00,0.000000,0.000000,0.000000,0.000000,...,0,34.25,3.0,2.5,1.0,1.0,1.0,False,True,19.0
5,98.245526,0.944722,0.000000,31.0,1,2018-06-02 20:15:26.315000064-04:00,0.000000,0.000000,0.000000,0.000000,...,0,33.50,3.0,2.5,1.0,1.0,1.0,False,True,19.0
6,96.862807,0.228998,0.098002,23.0,1,2018-06-02 21:15:26.089999872-04:00,0.000000,0.000000,0.000000,0.000000,...,0,22.86,3.0,3.0,0.0,0.0,1.5,False,True,20.0
7,97.967464,1.161249,0.322936,1.0,1,2018-06-02 22:15:27.040999936-04:00,0.000000,0.000000,0.000000,0.000000,...,0,42.77,3.0,2.5,0.0,0.0,1.5,True,True,21.0
8,97.758202,0.080504,0.000000,6.0,1,2018-06-03 11:56:42.046000128-04:00,0.542322,0.000000,0.000000,0.000000,...,0,20.83,3.0,3.0,0.0,2.0,2.0,False,True,11.0
9,97.731224,16.479991,-7.862508,5.0,3,2018-06-03 16:15:35.526000128-04:00,0.000000,0.857009,0.636780,0.000000,...,0,69.28,3.5,3.5,0.0,0.0,2.0,True,True,16.0


In [70]:
y_mood = finalMergedDF["mood"]
y_morale = finalMergedDF["morale"]
y_stress = finalMergedDF["stress"]
y_social = finalMergedDF["unique_interactions"]
y_fatigue = finalMergedDF["fatigue"]
y_compulsions = finalMergedDF["compulsions"]
y_alone = finalMergedDF["alone"]

finalFeaturesList = list(finalMergedDF.columns.drop('time').drop("mood").drop("morale").drop("stress").drop("unique_interactions").drop("fatigue").drop("compulsions").drop("alone"))
X = finalMergedDF[finalFeaturesList]

# mood
X_train, X_test, y_mood_train, y_mood_test = train_test_split(
    X, y_mood, test_size=0.20, shuffle=False)

mood_LM = linear_model.LinearRegression()
mood_LM = mood_LM.fit(X_train, y_mood_train)

# morale
X_train, X_test, y_morale_train, y_morale_test = train_test_split(
    X, y_morale, test_size=0.20, shuffle=False)

morale_LM = SVR(C=0.1, gamma=0.001)
morale_LM = morale_LM.fit(X_train, y_morale_train)

# stress
X_train, X_test, y_stress_train, y_stress_test = train_test_split(
    X, y_stress, test_size=0.20, shuffle=False)

stress_LM = linear_model.LinearRegression()
stress_LM = stress_LM.fit(X_train, y_stress_train)

# social
X_train, X_test, y_social_train, y_social_test = train_test_split(
    X, y_social, test_size=0.20, shuffle=False)

social_LM = linear_model.LinearRegression()
social_LM = social_LM.fit(X_train, y_social_train)

# fatigue
X_train, X_test, y_fatigue_train, y_fatigue_test = train_test_split(
    X, y_fatigue, test_size=0.20, shuffle=False)

fatigue_LM = SVR(C=0.1, gamma=0.001)
fatigue_LM = fatigue_LM.fit(X_train, y_fatigue_train)

# compulsions
X_train, X_test, y_compulsions_train, y_compulsions_test = train_test_split(
    X, y_compulsions, test_size=0.20, shuffle=False)

compulsions_LM = linear_model.LinearRegression()
compulsions_LM = compulsions_LM.fit(X_train, y_compulsions_train)

# alone
X_train, X_test, y_alone_train, y_alone_test = train_test_split(
    X, y_alone, test_size=0.20, shuffle=False)

alone_LM = linear_model.LinearRegression()
alone_LM = alone_LM.fit(X_train, y_alone_train)

print(mood_LM.score(X_test, y_mood_test))
print(morale_LM.score(X_test, y_morale_test))
print(stress_LM.score(X_test, y_stress_test))
print(social_LM.score(X_test, y_social_test))
print(fatigue_LM.score(X_test, y_fatigue_test))
print(compulsions_LM.score(X_test, y_compulsions_test))
print(alone_LM.score(X_test, y_alone_test))

-0.0973570432543
0.00853351210071
-0.408510237465
-0.110592530912
-0.21312322826
-0.189664838797
-0.112711138285


In [ ]:
print(finalFeaturesList)

In [ ]:
# mood: train
from sklearn.externals import joblib


# productivity-mood .018464572574345972
# X_productivity_train, X_productivity_test, y_productivity_mood_train, y_productivity_mood_test = train_test_split(
#     X_productivity, y_productivity_mood, test_size=0.20, shuffle=False)

X_productivity = productivityOuputsDF[productivityFeaturesList]
y_productivity_mood = productivityOuputsDF["mood"]

productivity_mood_SVR = SVR(C=1, gamma=0.0001)
productivity_mood_SVR = productivity_mood_SVR.fit(X_productivity, y_productivity_mood)

joblib.dump(productivity_mood_SVR, '/Users/jxu2/github/thesis_explorations/life_support/predictor/productivity_mood_SVR.pkl') 


# affectiva-mood 0.0533168765421
# X_affectiva_train, X_affectiva_test, y_affectiva_mood_train, y_affectiva_mood_test = train_test_split(
#     X_affectiva, y_affectiva_mood, test_size=0.20, shuffle=False)

affectiva_mood_LM = linear_model.LinearRegression()
affectiva_mood_LM = affectiva_mood_LM.fit(X_affectiva, y_affectiva_mood)

joblib.dump(affectiva_mood_LM, '/Users/jxu2/github/thesis_explorations/life_support/predictor/affectiva_mood_LM.pkl') 


# stepCount-mood 0.022634405871709836
# X_stepCount_train, X_stepCount_test, y_stepCount_mood_train, y_stepCount_mood_test = train_test_split(
#     X_stepCount, y_stepCount_mood, test_size=0.20, shuffle=False)

stepCount_mood_SVR = SVR(C=0.1, gamma=1)
stepCount_mood_SVR = stepCount_mood_SVR.fit(X_stepCount_train, y_stepCount_mood_train)

joblib.dump(stepCount_mood_SVR, '/Users/jxu2/github/thesis_explorations/life_support/predictor/stepCount_mood_SVR.pkl') 


# test

y_productivity_mood = finalMergedDF["mood"]

#productivity-mood
productivityFeaturesList = list(productivityDF.columns.drop('time'))
X_productivity_merged = finalMergedDF[productivityFeaturesList]

# X_productivity_train, X_productivity_test, y_productivity_mood_train, y_productivity_mood_test = train_test_split(
#     X_productivity, y_productivity_mood, test_size=0.30, shuffle=False)

y_productivity_mood_pred = productivity_mood_SVR.predict(X_productivity_merged)


#affectiva-mood
affectivaFeaturesList = list(affectivaDF.columns.drop('time'))
X_affectiva_merged = finalMergedDF[affectivaFeaturesList]

# X_productivity_train, X_productivity_test, y_productivity_mood_train, y_productivity_mood_test = train_test_split(
#     X_productivity, y_productivity_mood, test_size=0.30, shuffle=False)

y_affectiva_mood_pred = affectiva_mood_LM.predict(X_affectiva_merged)


#stepCount-mood
stepCountFeaturesList = list(stepCountDF.columns.drop('time'))
X_stepCount_merged = finalMergedDF[stepCountFeaturesList]

# X_productivity_train, X_productivity_test, y_productivity_mood_train, y_productivity_mood_test = train_test_split(
#     X_productivity, y_productivity_mood, test_size=0.30, shuffle=False)

y_stepCount_mood_pred = stepCount_mood_SVR.predict(X_stepCount_merged)


In [ ]:

#ensemble model

ensembleMoodDF = pd.DataFrame()
ensembleMoodDF['productivity_predictions'] = y_productivity_mood_pred
ensembleMoodDF['affectiva_predictions'] = y_affectiva_mood_pred
ensembleMoodDF['stepCount_predictions'] = y_stepCount_mood_pred
ensembleMoodDF['actual_mood_scores'] = y_productivity_mood

ensembleFeaturesList = list(ensembleMoodDF.columns.drop('actual_mood_scores'))
X_ensemble = ensembleMoodDF[ensembleFeaturesList]
y_ensemble_mood = ensembleMoodDF["actual_mood_scores"]

ensemble_mood_SVR = SVR(C=0.17, gamma=12)
ensemble_mood_SVR = ensemble_mood_SVR.fit(X_ensemble, y_ensemble_mood)

joblib.dump(ensemble_mood_SVR, '/Users/jxu2/github/thesis_explorations/life_support/predictor/saved_models/ensemble_mood_SVR.pkl')

# ensemble_mood_LM = linear_model.LinearRegression()
# ensemble_mood_LM = ensemble_mood_LM.fit(X_ensemble_train, y_ensemble_mood_train)

# ensemble_mood_LM.score(X_ensemble_test, y_ensemble_mood_test)

# ensemble_mood_RF = ensemble.RandomForestRegressor(bootstrap=True, max_features='sqrt', max_depth=100, min_samples_leaf=5, min_samples_split=9, n_estimators=600)
# ensemble_mood_RF = ensemble_mood_RF.fit(X_ensemble_train, y_ensemble_mood_train)

# ensemble_mood_RF.score(X_ensemble_test, y_ensemble_mood_test)

In [ ]:
loaded_productivity_mood_SVR = joblib.load('/Users/jxu2/github/thesis_explorations/life_support/predictor/saved_models/productivity_mood_SVR.pkl')

loaded_productivity_mood_SVR.predict(productivityDF.iloc[-1]['productivity_score'])

In [ ]:
ensembleMoodDF = pd.DataFrame()
ensembleMoodDF['productivity_predictions'] = y_productivity_mood_pred
ensembleMoodDF['affectiva_predictions'] = y_affectiva_mood_pred
ensembleMoodDF['stepCount_predictions'] = y_stepCount_mood_pred
ensembleMoodDF['actual_mood_scores'] = y_productivity_mood

ensembleFeaturesList = list(ensembleMoodDF.columns.drop('actual_mood_scores'))
X_ensemble = ensembleMoodDF[ensembleFeaturesList]
y_ensemble_mood = ensembleMoodDF["actual_mood_scores"]

X_ensemble_train, X_ensemble_test, y_ensemble_mood_train, y_ensemble_mood_test = train_test_split(
    X_ensemble, y_ensemble_mood, test_size=0.20, shuffle=False)

ensemble_mood_SVR = SVR(C=0.17, gamma=12)
ensemble_mood_SVR = ensemble_mood_SVR.fit(X_ensemble_train, y_ensemble_mood_train)
ensemble_mood_SVR.score(X_ensemble_test, y_ensemble_mood_test)

In [ ]:
Cs = [0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25]
gammas = [11, 12, 13, 14, 15, 16]
param_grid_SVC = {'C': Cs, 'gamma' : gammas}

svc_grid_search = GridSearchCV(estimator = ensemble_mood_SVR, param_grid = param_grid_SVC, 
                          cv = 3, n_jobs = -1, verbose = 2)
# Fit the grid search to the data
svc_grid_search.fit(X_ensemble_train, y_ensemble_mood_train)

In [ ]:
svc_grid_search.best_params_

In [ ]:
# morale: train

y_productivity_morale = productivityOuputsDF["morale"]

# productivity-morale 0.137327400617
# X_productivity_train, X_productivity_test, y_morale_train, y_morale_test = train_test_split(
#     X_productivity, y_morale, test_size=0.20, shuffle=False)

productivity_morale_LM = linear_model.LinearRegression()
productivity_morale_LM = productivity_morale_LM.fit(X_productivity, y_productivity_morale)


# affectiva-morale 0.0577677711737
# X_affectiva_train, X_affectiva_test, y_morale_train, y_morale_test = train_test_split(
#     X_affectiva, y_morale, test_size=0.20, shuffle=False)

y_affectiva_morale = affectivaOuputsDF["morale"]

affectiva_morale_LM = linear_model.LinearRegression()
affectiva_morale_LM = affectiva_morale_LM.fit(X_affectiva, y_affectiva_morale)


# test

y_productivity_morale = finalMergedDF["morale"]

#productivity-morale
# X_productivity_train, X_productivity_test, y_productivity_mood_train, y_productivity_mood_test = train_test_split(
#     X_productivity, y_productivity_mood, test_size=0.30, shuffle=False)

y_productivity_morale_pred = productivity_morale_LM.predict(X_productivity_merged)


#affectiva-morale
# X_productivity_train, X_productivity_test, y_productivity_mood_train, y_productivity_mood_test = train_test_split(
#     X_productivity, y_productivity_mood, test_size=0.30, shuffle=False)

y_affectiva_morale_pred = affectiva_morale_LM.predict(X_affectiva_merged)



#ensemble model

ensembleMoraleDF = pd.DataFrame()
ensembleMoraleDF['productivity_predictions'] = y_productivity_morale_pred
ensembleMoraleDF['affectiva_predictions'] = y_affectiva_morale_pred
ensembleMoraleDF['actual_morale_scores'] = y_productivity_morale

ensembleMoraleFeaturesList = list(ensembleMoraleDF.columns.drop('actual_morale_scores'))
X_ensemble_morale = ensembleMoraleDF[ensembleMoraleFeaturesList]
y_ensemble_morale = ensembleMoraleDF["actual_morale_scores"]

X_ensemble_morale_train, X_ensemble_morale_test, y_ensemble_morale_train, y_ensemble_morale_test = train_test_split(
    X_ensemble_morale, y_ensemble_morale, test_size=0.20, shuffle=False)


ensemble_morale_LM = linear_model.LinearRegression()
ensemble_morale_LM = ensemble_morale_LM.fit(X_ensemble_morale_train, y_ensemble_morale_train)

ensemble_morale_LM.score(X_ensemble_morale_test, y_ensemble_morale_test)

# ensemble_morale_RF = ensemble.RandomForestRegressor(bootstrap=True, max_features='sqrt', max_depth=100, min_samples_leaf=5, min_samples_split=190, n_estimators=600)
# ensemble_morale_RF = ensemble_morale_RF.fit(X_ensemble_morale_train, y_ensemble_morale_train)

# ensemble_morale_RF.score(X_ensemble_morale_test, y_ensemble_morale_test)

In [ ]:
# social

# affectiva-social 0.00225508115628
X_affectiva_train, X_affectiva_test, y_social_train, y_social_test = train_test_split(
    X_affectiva, y_social, test_size=0.20, shuffle=False)

affectiva_social_LM = linear_model.LinearRegression()
affectiva_social_LM = affectiva_social_LM.fit(X_affectiva_train, y_social_train)

# stepCount-social 0.0891346425851
X_stepCount_train, X_stepCount_test, y_social_train, y_social_test = train_test_split(
    X_stepCount, y_social, test_size=0.20, shuffle=False)

stepCount_social_LM = SVR(C=0.1, gamma=0.001)
stepCount_social_LM = stepCount_social_LM.fit(X_stepCount_train, y_social_train)
